# Đọc dữ liệu và chuẩn hóa tên cột
Notebook này đọc 2 file Excel trong thư mục `data/`:
- `Distributors (1).xlsx`: danh sách hàng nhập
- `Suppliers (1).xlsx`: danh sách hàng xuất

Sau đó chuẩn hóa tên cột (loại bỏ dấu, khoảng trắng, ký tự đặc biệt; dùng chữ thường và `_`).

In [3]:
# Import libraries and helper to standardize column names
import pandas as pd
import re
import unicodedata

def normalize_text(s: str) -> str:
    # Strip, NFC normalize, remove accents, lowercase
    s = s.strip()
    s = unicodedata.normalize('NFKD', s)
    s = ''.join(c for c in s if not unicodedata.combining(c))
    s = s.lower()
    # Replace non-alphanumeric with underscores and collapse repeats
    s = re.sub(r"[^a-z0-9]+", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s

def standardize_columns(df: pd.DataFrame, mapping: dict | None = None) -> pd.DataFrame:
    # First normalize all columns generically
    new_cols = {col: normalize_text(str(col)) for col in df.columns}
    df = df.rename(columns=new_cols)
    # Optional specific mapping to canonical names (user can extend)
    if mapping:
        df = df.rename(columns=mapping)
    return df

In [4]:
# Read Excel files
distributors_path = "data/Distributors (1).xlsx"
suppliers_path = "data/Suppliers (1).xlsx"

df_import = pd.read_excel(distributors_path)  # hàng nhập
df_export = pd.read_excel(suppliers_path)    # hàng xuất

print("Shapes:", df_import.shape, df_export.shape)

Shapes: (260, 7) (159, 7)


In [6]:
# Standardize column names and preview
# Optional: add explicit mapping if needed, e.g.:
# mapping_import = {"ma_sp": "product_code", "ten_sp": "product_name"}
# mapping_export = {"ma_sp": "product_code", "ten_khach": "customer_name"}
mapping_import = None
mapping_export = None

df_import = standardize_columns(df_import, mapping_import)
df_export = standardize_columns(df_export, mapping_export)

print("Import columns:", list(df_import.columns))
print("Export columns:", list(df_export.columns))

display(df_import.head())
display(df_export.head())

Import columns: ['stt', 'loai_khai_bao', 'xuat_su', 'thuong_pham', 'thoi_gian_thuc_hien', 'nguoi_thuc_hien', 'thoi_gian_tao']
Export columns: ['stt', 'loai_khai_bao', 'xuat_su', 'thuong_pham', 'thoi_gian_thuc_hien', 'nguoi_thuc_hien', 'thoi_gian_tao']


,stt,loai_khai_bao,xuat_su,thuong_pham,thoi_gian_thuc_hien,nguoi_thuc_hien,thoi_gian_tao
0,1,Phân Phối Sản phẩm nhập khẩu,Sản phẩm nhập khẩu,Lê,04/12/2025 14:26:29,DƯƠNG MINH CƯỜNG,04/12/2025 14:27:04
1,2,Phân Phối Sản phẩm nội địa,Sản phẩm nội địa,Bắp cải trắng,04/12/2025 12:56:25,NGUYỄN THÀNH LONG,04/12/2025 12:56:49
2,3,Phân Phối Sản phẩm nội địa,Sản phẩm nội địa,Dưa lưới,04/12/2025 10:20:31,NGUYỄN THỊ BÍCH VÂN,04/12/2025 10:21:02
3,4,Phân Phối Sản phẩm nội địa,Sản phẩm nội địa,Dưa lưới,04/12/2025 10:19:54,NGUYỄN THỊ BÍCH VÂN,04/12/2025 10:20:22
4,5,Phân Phối Sản phẩm nội địa,Sản phẩm nội địa,Dưa lưới,04/12/2025 10:19:02,NGUYỄN THỊ BÍCH VÂN,04/12/2025 10:19:47


,stt,loai_khai_bao,xuat_su,thuong_pham,thoi_gian_thuc_hien,nguoi_thuc_hien,thoi_gian_tao
0,1,Cung Ứng Sản phẩm nhập khẩu,Sản phẩm nhập khẩu,Lê,04/12/2025 14:24:54,DƯƠNG MINH CƯỜNG,04/12/2025 14:26:26
1,2,Cung Ứng Sản phẩm nội địa,Sản phẩm nội địa,Bắp cải trắng,04/12/2025 09:18:43,NGUYỄN THÀNH LONG,04/12/2025 12:56:23
2,3,Cung Ứng Sản phẩm nội địa,Sản phẩm nội địa,Dưa lưới,04/12/2025 09:52:53,NGUYỄN THỊ BÍCH VÂN,04/12/2025 09:53:19
3,4,Cung Ứng Sản phẩm nội địa,Sản phẩm nội địa,Dưa lưới,04/12/2025 09:52:25,NGUYỄN THỊ BÍCH VÂN,04/12/2025 09:52:49
4,5,Cung Ứng Sản phẩm nội địa,Sản phẩm nội địa,Cải xanh,04/12/2025 05:54:01,Vũ Phước Chân,04/12/2025 05:58:21


In [7]:
# Drop specified columns and map names to Kiot
drop_cols = ["loai_khai_bao", "thoi_gian_thuc_hien", "thuong_pham", "xuat_su"]

name_to_kiot_raw = {
    "TĂNG CẨM HẰNG": "T1-015",
    "NGUYỄN THỊ NGỌC SƯƠNG": "TS-010",
    "NGUYỄN THỊ BÍCH VÂN": "T2-045",
    "NGUYỄN HOÀNG NAM": "T1-019",
    "DƯƠNG MINH CƯỜNG": "T2-079",
    "ĐẶNG THỊ THÙY DƯƠNG": "NA-040-041",
    "PHẠM THỊ NGỌC THÚY": "NA-033-034",
    "NGUYỄN THỊ KIM LOAN": "NA-021-022",
    "NGUYỄN THÀNH LONG": "B1-091",
    "HÀ QUỐC VINH": "B1-093",
    "NGUYỄN THỊ NƯƠNG": "B2-124",
    "VŨ PHƯỚC CHÂN": "B3-025",
    "NGUYỄN NGỌC SANG": "B3-063",
}

def normalize_name(s):
    if pd.isna(s):
        return None
    s = str(s).strip()
    s = unicodedata.normalize('NFKD', s)
    s = ''.join(c for c in s if not unicodedata.combining(c))
    s = s.upper()
    s = re.sub(r"\s+", " ", s)
    return s

# Build normalized mapping for robust matching (ignoring accents/case)
name_to_kiot = {normalize_name(k): v for k, v in name_to_kiot_raw.items()}

def apply_mapping(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    # Add 'kiot' by normalized matching on 'nguoi_thuc_hien'
    norm = out["nguoi_thuc_hien"].map(normalize_name)
    out["kiot"] = norm.map(name_to_kiot)
    # Drop the requested columns if present
    to_drop = [c for c in drop_cols if c in out.columns]
    out = out.drop(columns=to_drop)
    # Report any names not mapped
    missing = out.loc[out["kiot"].isna(), "nguoi_thuc_hien"].dropna().unique().tolist()
    if missing:
        print("Unmapped names (please add to mapping):", missing)
    # Reorder columns: stt, nguoi_thuc_hien, kiot, thoi_gian_tao (if present)
    prefer = [c for c in ["stt", "nguoi_thuc_hien", "kiot", "thoi_gian_tao"] if c in out.columns]
    others = [c for c in out.columns if c not in prefer]
    out = out[prefer + others]
    return out

df_import_mapped = apply_mapping(df_import)
df_export_mapped = apply_mapping(df_export)

print("Import mapped columns:", list(df_import_mapped.columns))
print("Export mapped columns:", list(df_export_mapped.columns))

display(df_import_mapped.head())
display(df_export_mapped.head())

Unmapped names (please add to mapping): ['Nguyễn Anh Tuấn', 'Thịnh Nguyễn', 'Hoàng Minh ']
Unmapped names (please add to mapping): ['Nguyễn Anh Tuấn', 'Thịnh Nguyễn', 'Declan Rice ', 'Hoàng Minh ']
Import mapped columns: ['stt', 'nguoi_thuc_hien', 'kiot', 'thoi_gian_tao']
Export mapped columns: ['stt', 'nguoi_thuc_hien', 'kiot', 'thoi_gian_tao']


,stt,nguoi_thuc_hien,kiot,thoi_gian_tao
0,1,DƯƠNG MINH CƯỜNG,T2-079,04/12/2025 14:27:04
1,2,NGUYỄN THÀNH LONG,B1-091,04/12/2025 12:56:49
2,3,NGUYỄN THỊ BÍCH VÂN,T2-045,04/12/2025 10:21:02
3,4,NGUYỄN THỊ BÍCH VÂN,T2-045,04/12/2025 10:20:22
4,5,NGUYỄN THỊ BÍCH VÂN,T2-045,04/12/2025 10:19:47


,stt,nguoi_thuc_hien,kiot,thoi_gian_tao
0,1,DƯƠNG MINH CƯỜNG,T2-079,04/12/2025 14:26:26
1,2,NGUYỄN THÀNH LONG,B1-091,04/12/2025 12:56:23
2,3,NGUYỄN THỊ BÍCH VÂN,T2-045,04/12/2025 09:53:19
3,4,NGUYỄN THỊ BÍCH VÂN,T2-045,04/12/2025 09:52:49
4,5,Vũ Phước Chân,B3-025,04/12/2025 05:58:21


In [8]:
# Build daily totals (1–15 Dec) for 13 merchants with 16:00 cutoff
import numpy as np
from datetime import datetime

# Canonical merchant list from the mapping keys
MERCHANTS_13 = list(name_to_kiot_raw.keys())

def parse_datetime_col(df: pd.DataFrame, col: str) -> pd.Series:
    # Parse day-first timestamps like '04/12/2025 14:27:04'
    return pd.to_datetime(df[col], dayfirst=True, errors='coerce')

def add_bucket_date(df: pd.DataFrame, ts_col: str, bucket_col: str = "bucket_date") -> pd.DataFrame:
    out = df.copy()
    ts = parse_datetime_col(out, ts_col)
    # Day label is defined as (t - 16h).floor('D') + 1 day
    out[bucket_col] = (ts - pd.Timedelta(hours=16)).dt.floor('D') + pd.Timedelta(days=1)
    out[bucket_col] = out[bucket_col].dt.date
    return out

# Normalize to canonical merchant names (restrict to the provided 13)
canon_map = {normalize_name(k): k for k in MERCHANTS_13}
def assign_canonical_merchant(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    norm = out["nguoi_thuc_hien"].map(normalize_name)
    out["merchant"] = norm.map(canon_map)
    # Keep only rows that map to our 13 merchants
    out = out[out["merchant"].notna()]
    return out

# Prepare frames with bucketed dates and merchant labels
imp = assign_canonical_merchant(add_bucket_date(df_import_mapped, "thoi_gian_tao"))
exp = assign_canonical_merchant(add_bucket_date(df_export_mapped, "thoi_gian_tao"))

# Determine year from data if possible; fallback to current year
def pick_year(*frames):
    for f in frames:
        s = pd.to_datetime(f.get("thoi_gian_tao"), dayfirst=True, errors='coerce')
        yr = s.dt.year.dropna().astype(int)
        if len(yr):
            return int(yr.iloc[0])
    return datetime.now().year

year = pick_year(imp, exp)

# Build date range 1–15 Dec of that year
date_range = pd.date_range(start=datetime(year, 12, 1), end=datetime(year, 12, 15), freq='D').date

# Create full grid merchant x date
idx = pd.MultiIndex.from_product([MERCHANTS_13, date_range], names=["merchant", "date"])
base = pd.DataFrame(index=idx).reset_index()

# Aggregate counts per bucket date
imp_cnt = imp.groupby(["merchant", "bucket_date"], dropna=False).size().rename("total_import").reset_index().rename(columns={"bucket_date": "date"})
exp_cnt = exp.groupby(["merchant", "bucket_date"], dropna=False).size().rename("total_sell").reset_index().rename(columns={"bucket_date": "date"})

# Merge to full grid and fill missing with zeros
df_daily_totals = base.merge(imp_cnt, on=["merchant", "date"], how="left").merge(exp_cnt, on=["merchant", "date"], how="left")
df_daily_totals[["total_import", "total_sell"]] = df_daily_totals[["total_import", "total_sell"]].fillna(0).astype(int)

# Sort for readability
df_daily_totals = df_daily_totals.sort_values(["merchant", "date"]).reset_index(drop=True)

print(f"Year used: {year}")
print(df_daily_totals.head(10))
display(df_daily_totals)

Year used: 2025
           merchant        date  total_import  total_sell
0  DƯƠNG MINH CƯỜNG  2025-12-01             0           0
1  DƯƠNG MINH CƯỜNG  2025-12-02             0           1
2  DƯƠNG MINH CƯỜNG  2025-12-03             0           0
3  DƯƠNG MINH CƯỜNG  2025-12-04             1           1
4  DƯƠNG MINH CƯỜNG  2025-12-05             0           0
5  DƯƠNG MINH CƯỜNG  2025-12-06             0           0
6  DƯƠNG MINH CƯỜNG  2025-12-07             0           0
7  DƯƠNG MINH CƯỜNG  2025-12-08             0           0
8  DƯƠNG MINH CƯỜNG  2025-12-09             0           0
9  DƯƠNG MINH CƯỜNG  2025-12-10             0           0


,merchant,date,total_import,total_sell
0,DƯƠNG MINH CƯỜNG,2025-12-01,0,0
1,DƯƠNG MINH CƯỜNG,2025-12-02,0,1
2,DƯƠNG MINH CƯỜNG,2025-12-03,0,0
3,DƯƠNG MINH CƯỜNG,2025-12-04,1,1
4,DƯƠNG MINH CƯỜNG,2025-12-05,0,0
...,...,...,...,...
190,ĐẶNG THỊ THÙY DƯƠNG,2025-12-11,0,0
191,ĐẶNG THỊ THÙY DƯƠNG,2025-12-12,0,0
192,ĐẶNG THỊ THÙY DƯƠNG,2025-12-13,0,0
193,ĐẶNG THỊ THÙY DƯƠNG,2025-12-14,0,0


In [9]:
# Totals for 2025-12-01 (16:00 cutoff windows)
import pandas as pd
from datetime import datetime

date_target = pd.to_datetime("2025-12-01").date()
assert 'df_daily_totals' in globals(), "df_daily_totals not found. Please run previous cells."
df_day = df_daily_totals[df_daily_totals["date"] == date_target].copy()

totals = df_day[["total_import", "total_sell"]].sum().to_frame(name="total").T
totals.index = [str(date_target)]

print(f"Date (cutoff 16:00): {date_target}")
display(totals)

# Optional: show per-merchant breakdown for that day
display(df_day.sort_values("merchant").reset_index(drop=True))

Date (cutoff 16:00): 2025-12-01


,total_import,total_sell
2025-12-01,16,6


,merchant,date,total_import,total_sell
0,DƯƠNG MINH CƯỜNG,2025-12-01,0,0
1,HÀ QUỐC VINH,2025-12-01,0,0
2,NGUYỄN HOÀNG NAM,2025-12-01,0,0
3,NGUYỄN NGỌC SANG,2025-12-01,0,2
4,NGUYỄN THÀNH LONG,2025-12-01,0,0
5,NGUYỄN THỊ BÍCH VÂN,2025-12-01,10,1
6,NGUYỄN THỊ KIM LOAN,2025-12-01,0,0
7,NGUYỄN THỊ NGỌC SƯƠNG,2025-12-01,0,0
8,NGUYỄN THỊ NƯƠNG,2025-12-01,0,0
9,PHẠM THỊ NGỌC THÚY,2025-12-01,2,1
